In [98]:
import numpy as np
from keras.layers import Conv1D, LSTM, Dense, TimeDistributed, Embedding, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Bidirectional, Dropout
import random

# Load and preprocess data
def load_data(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        words = file.read().splitlines()
    return words

def simulate_missing_letters(word, p_missing=0.4):
    """
    Randomly replace letters with '0' to simulate missing letters,
    with p_missing chance for each letter.
    """
    simulated_word = ''
    for char in word:
        simulated_word += '0' if random.random() < p_missing else char
    return simulated_word

def preprocess_data(words, char_to_int, max_word_length, p_missing=0.4):
    """
    Preprocess data to create training examples with simulated missing letters.
    """
    sequences = []
    targets = []
    for word in words:
        word_with_missing = simulate_missing_letters(word, p_missing)
        sequences.append([char_to_int[char] if char in char_to_int else char_to_int['0'] for char in word_with_missing])
        targets.append([char_to_int[char] for char in word])
    # Pad sequences for consistent input size
    sequences = pad_sequences(sequences, maxlen=max_word_length, padding='post')
    targets = pad_sequences(targets, maxlen=max_word_length, padding='post')
    return sequences, targets


file_path = 'words_250000_train.txt'

words = load_data(file_path)

chars = sorted(list(set(''.join(words) + '0')))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
max_word_length = 45

X, y = preprocess_data(words, char_to_int, max_word_length)
y = to_categorical(y, num_classes=len(chars))

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(input_dim=len(chars), output_dim=64, trainable=True))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.001))))
model.add(Dropout(0.4))
model.add(TimeDistributed(Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min', restore_best_weights=True)

model.fit(X, np.array(y), validation_split=0.2, epochs=10, batch_size=128, callbacks=[early_stopping])

Epoch 1/10
1421/1421 [==============================] - 702s 489ms/step - loss: 0.3760 - accuracy: 0.9027 - val_loss: 0.2586 - val_accuracy: 0.9294
Epoch 2/10
1421/1421 [==============================] - 668s 470ms/step - loss: 0.2377 - accuracy: 0.9338 - val_loss: 0.2487 - val_accuracy: 0.9312
Epoch 3/10
1421/1421 [==============================] - 634s 447ms/step - loss: 0.2262 - accuracy: 0.9358 - val_loss: 0.2448 - val_accuracy: 0.9327
Epoch 4/10
1421/1421 [==============================] - 615s 433ms/step - loss: 0.2199 - accuracy: 0.9371 - val_loss: 0.2403 - val_accuracy: 0.9335
Epoch 5/10
1421/1421 [==============================] - 609s 429ms/step - loss: 0.2154 - accuracy: 0.9382 - val_loss: 0.2400 - val_accuracy: 0.9347
Epoch 6/10
1421/1421 [==============================] - 652s 459ms/step - loss: 0.2121 - accuracy: 0.9390 - val_loss: 0.2425 - val_accuracy: 0.9343
Epoch 7/10
1421/1421 [==============================] - 510s 359ms/step - loss: 0.2094 - accuracy: 0.9396 - val_

In [3]:
from keras.models import load_model

# model.save('lstm_model_bi_5.h5')
model = load_model('lstm_model_bi_3.h5')
# x = load_model(r"C:\Users\Fatima Azfar\Downloads\model.h5")

In [112]:
def predict_word(model, word_with_missing, char_to_int, int_to_char, max_word_length):
    while '0' in word_with_missing:
        word_encoded = [char_to_int[char] if char in char_to_int else char_to_int['0'] for char in word_with_missing]
        word_padded = pad_sequences([word_encoded], maxlen=max_word_length, padding='post')
        
        prediction = model.predict(word_padded)
        predict_vector = prediction[0]
        
        for i, char in enumerate(word_with_missing):
            if char == '0':
                predicted_char_index = np.argmax(predict_vector[i])
                word_with_missing = word_with_missing[:i] + int_to_char[predicted_char_index] + word_with_missing[i+1:]
                break
                
    return word_with_missing

# Adjust for your testing
word_with_missing = 'ang_r'  # Example with missing letter 'e'
word_with_missing = word_with_missing.replace('_', '0')  # Replace '_' with '0' for consistency
predicted_word = predict_word(model, word_with_missing, char_to_int, int_to_char, max_word_length)
print(f"Original: {word_with_missing.replace('0', '_')}, Predicted: {predicted_word}")

1/1 [==============================] - 0s 25ms/step
Original: ang_r, Predicted: anger


In [138]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def predict_word(model, word_with_missing, char_to_int, int_to_char, max_word_length, guessed_letters):
    print("LSTM")
    word_encoded = [char_to_int[char] if char in char_to_int else char_to_int['0'] for char in word_with_missing]
    word_padded = pad_sequences([word_encoded], maxlen=max_word_length, padding='post')
    
    prediction = model.predict(word_padded)
    predict_vector = prediction[0]
    
    for i, char in enumerate(word_with_missing):
        if char == '0':
            probabilities = predict_vector[i]
            sorted_indices = np.argsort(-probabilities)
            for idx in sorted_indices:
                predicted_char = int_to_char[idx]
                if predicted_char != '0' and predicted_char not in guessed_letters:
                    return predicted_char
    return None  # Return None if all characters are guessed or no suitable character is found

# Example usage:
# model = YourModel()  # Assume you have a pre-trained model
word_with_missing = "an000"
# char_to_int = {'h': 1, 'e': 2, 'l': 3, '0': 0}  # Example character to integer mapping
# int_to_char = {1: 'h', 2: 'e', 3: 'l', 0: '0'}  # Example integer to character mapping
# max_word_length = 5
guessed_letters = set(['e','o','i','s','t','d','a'])  # Example set of already guessed letters
predicted_char = predict_word(model, word_with_missing, char_to_int, int_to_char, max_word_length, guessed_letters)
print(predicted_char)

LSTM
1/1 [==============================] - 1s 1s/step
g


In [88]:
def load_words(filepath):
    with open(filepath, 'r') as file:
        words = file.read().splitlines()
    words_by_length = {}
    for word in words:
        length = len(word)
        if length in words_by_length:
            words_by_length[length].append(word)
        else:
            words_by_length[length] = [word]
    return words_by_length

from collections import Counter

def update_possible_words(possible_words, guessed_letter, feedback, target_word):
    """
    Update the list of possible words based on the feedback of the guessed letter.
    """
    new_possible_words = []
    if feedback:  # Correct guess
        for word in possible_words:
            if all((c == guessed_letter or c == '_' or c in target_word) if g == guessed_letter else c != guessed_letter for c, g in zip(word, target_word)):
                new_possible_words.append(word)
    else:  # Incorrect guess
        new_possible_words = [word for word in possible_words if guessed_letter not in word]
    return new_possible_words

def get_positional_letter_frequencies(words, guessed_letters):
    """
    Calculate letter frequencies based on their positions within words.
    """
    positional_frequencies = [Counter() for _ in range(len(words[0]))]
    for word in words:
        for index, letter in enumerate(word):
            if letter not in guessed_letters:
                positional_frequencies[index][letter] += 1
    return positional_frequencies

def get_best_guess(positional_frequencies, guessed_letters):
    """
    Determine the best letter to guess next based on positional frequencies and letter distribution.
    """
    overall_frequencies = Counter()
    for position in positional_frequencies:
        for letter, frequency in position.items():
            overall_frequencies[letter] += frequency
    most_common = [letter for letter, _ in overall_frequencies.most_common() if letter not in guessed_letters]
    return most_common[0] if most_common else None

def guess(guessed_letters, possible_words):
    # Calculate positional letter frequencies
    positional_frequencies = get_positional_letter_frequencies(possible_words, guessed_letters)
    
    # Get the best guess
    best_guess = get_best_guess(positional_frequencies, guessed_letters)
    
    return best_guess

def simulate_hangman_game(target_word):
    possible_words = load_words(r'C:\Users\Fatima Azfar\Documents\assignment-gigs\hangman-ai\words_250000_train.txt')[len(target_word)]
    guessed_letters = set()
    remaining_letters = set(target_word)  # Track letters still needed to be guessed correctly

    tries = 6  # Limit the player to 6 tries

    while remaining_letters and tries > 0:
        positional_frequencies = get_positional_letter_frequencies(possible_words, guessed_letters)
        guess_letter = get_best_guess(positional_frequencies, guessed_letters)
        if not guess_letter:  # If there are no letters left to guess, exit the loop
            print("Ran out of letters to guess.")
            break

        guessed_letters.add(guess_letter)
        print(f"Guessing letter: {guess_letter}")

        if guess_letter in target_word:
            print("Correct guess!")
            possible_words = update_possible_words(possible_words, guess_letter, True, target_word)
            remaining_letters.discard(guess_letter)
        else:
            print("Incorrect guess.")
            possible_words = update_possible_words(possible_words, guess_letter, False, target_word)
            tries -= 1

        current_state = ''.join([letter if letter in guessed_letters else '_' for letter in target_word])
        print(f"Current word state: {current_state}")
        print(f"Tries remaining: {tries}")

    if not remaining_letters:
        print(f"Congratulations! You've guessed the word: {target_word}")
    else:
        print(f"Game over. The word was: {target_word}")
        
def load_words_data(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        words = file.read().splitlines()
    return words

words = load_words_data(r'C:\Users\Fatima Azfar\Documents\assignment-gigs\hangman-ai\words_250000_train.txt')
import random
# simulate_hangman_game("bastardization")
for i in range(5):
    word = random.choice(words)
    simulate_hangman_game(word)
    print("--------------------------")

Guessing letter: e
Correct guess!
Current word state: _e____e__e_e__
Tries remaining: 6
Guessing letter: i
Correct guess!
Current word state: _e__i_e__e_e__
Tries remaining: 6
Guessing letter: s
Correct guess!
Current word state: _e__i_e__e_e_s
Tries remaining: 6
Guessing letter: o
Correct guess!
Current word state: _e__i_eo_e_e_s
Tries remaining: 6
Guessing letter: r
Correct guess!
Current word state: _e__ireo_ere_s
Tries remaining: 6
Guessing letter: l
Correct guess!
Current word state: le__ireo_ere_s
Tries remaining: 6
Guessing letter: m
Correct guess!
Current word state: lem_ireo_ere_s
Tries remaining: 6
Guessing letter: a
Correct guess!
Current word state: lemaireo_ere_s
Tries remaining: 6
Guessing letter: c
Correct guess!
Current word state: lemaireocere_s
Tries remaining: 6
Guessing letter: u
Correct guess!
Current word state: lemaireocereus
Tries remaining: 6
Congratulations! You've guessed the word: lemaireocereus
--------------------------
Guessing letter: e
Correct guess!
Cu

# Conditional Probabilities

In [89]:
from collections import Counter, defaultdict
import random

def load_words(filepath):
    with open(filepath, 'r') as file:
        words = file.read().splitlines()
    words_by_length = {}
    for word in words:
        length = len(word)
        if length in words_by_length:
            words_by_length[length].append(word)
        else:
            words_by_length[length] = [word]
    return words_by_length

def calculate_conditional_probabilities(words):
    forward_probs = defaultdict(lambda: Counter())
    backward_probs = defaultdict(lambda: Counter())
    for word in words:
        for i in range(len(word) - 1):
            forward_probs[word[i]][word[i + 1]] += 1
            backward_probs[word[i + 1]][word[i]] += 1
    return forward_probs, backward_probs

def update_possible_words(possible_words, guessed_letter, feedback, target_word):
    new_possible_words = []
    if feedback:
        for word in possible_words:
            if guessed_letter in word:
                new_possible_words.append(word)
    else:
        new_possible_words = [word for word in possible_words if guessed_letter not in word]
    return new_possible_words

def get_positional_letter_frequencies(words, guessed_letters):
    positional_frequencies = [Counter() for _ in range(len(words[0]))]
    for word in words:
        for index, letter in enumerate(word):
            if letter not in guessed_letters:
                positional_frequencies[index][letter] += 1
    return positional_frequencies

def get_fallback_guess(guessed_letters):
    # List of letters in the English alphabet ordered by frequency (approximate)
    alphabet_by_frequency = 'etaoinsrhdlucmfywgpbvkxqjz'
    
    for letter in alphabet_by_frequency:
        if letter not in guessed_letters:
            return letter
    return None  # In case all letters have been guessed, which should not happen

def get_best_guess_with_fallback(masked_word, guessed_letters, forward_probs, backward_probs, positional_frequencies):
    # Attempt to use conditional probabilities first
    potential_guesses = Counter()
    letters_revealed = any(letter != '_' for letter in masked_word)

    if letters_revealed:
        for i, letter in enumerate(masked_word):
            if letter == '_':
                prev_letter = masked_word[i-1] if i > 0 else None
                next_letter = masked_word[i+1] if i < len(masked_word) - 1 else None
                
                if prev_letter and prev_letter != '_':
                    for letter, count in forward_probs[prev_letter].items():
                        potential_guesses[letter] += count
                if next_letter and next_letter != '_':
                    for letter, count in backward_probs[next_letter].items():
                        potential_guesses[letter] += count
        
        # Filter out already guessed letters
        for letter in guessed_letters:
            if letter in potential_guesses:
                del potential_guesses[letter]

        if potential_guesses:
            return potential_guesses.most_common(1)[0][0]

    # Fallback to original logic if no letters are revealed or conditional probabilities yield no result
    overall_frequencies = Counter()
    for position in positional_frequencies:
        for letter, frequency in position.items():
            overall_frequencies[letter] += frequency
    
    most_common = [letter for letter, _ in overall_frequencies.most_common() if letter not in guessed_letters]
    return most_common[0] if most_common else get_fallback_guess(guessed_letters)

def simulate_hangman_game(target_word, filepath):
    all_words = load_words(filepath)[len(target_word)]
    guessed_letters = set()
    remaining_letters = set(target_word)
    forward_probs, backward_probs = calculate_conditional_probabilities(all_words)
    positional_frequencies = get_positional_letter_frequencies(all_words, guessed_letters)
    tries = 6

    while remaining_letters and tries > 0:
        current_state = ''.join([letter if letter in guessed_letters else '_' for letter in target_word])
        guess_letter = get_best_guess_with_fallback(current_state, guessed_letters, forward_probs, backward_probs, positional_frequencies)
        
        if not guess_letter:
            print("Ran out of letters to guess.")
            break

        guessed_letters.add(guess_letter)
        print(f"Guessing letter: {guess_letter}")

        if guess_letter in target_word:
            print("Correct guess!")
            remaining_letters.discard(guess_letter)
        else:
            print("Incorrect guess.")
            tries -= 1
        
        print(f"Current word state: {current_state}")
        print(f"Tries remaining: {tries}")

    if not remaining_letters:
        print(f"Congratulations! You've guessed the word: {target_word}")
    else:
        print(f"Game over. The word was: {target_word}")

# Example usage
filepath = 'words_250000_train.txt'
for _ in range(1):
    simulate_hangman_game(random.choice(load_words(filepath)[len("example")]), filepath)
    print("--------------------------")

Guessing letter: e
Correct guess!
Current word state: _______
Tries remaining: 6
Guessing letter: r
Correct guess!
Current word state: _____e_
Tries remaining: 6
Guessing letter: i
Incorrect guess.
Current word state: r____er
Tries remaining: 5
Guessing letter: l
Correct guess!
Current word state: r____er
Tries remaining: 5
Guessing letter: a
Incorrect guess.
Current word state: r___ler
Tries remaining: 4
Guessing letter: o
Incorrect guess.
Current word state: r___ler
Tries remaining: 3
Guessing letter: u
Correct guess!
Current word state: r___ler
Tries remaining: 3
Guessing letter: n
Incorrect guess.
Current word state: ru__ler
Tries remaining: 2
Guessing letter: s
Incorrect guess.
Current word state: ru__ler
Tries remaining: 1
Guessing letter: t
Incorrect guess.
Current word state: ru__ler
Tries remaining: 0
Game over. The word was: ruffler
--------------------------


# Word embeddings model for fallback

In [90]:
# Statistical Model (used for fallback guesses)
## Letter Embeddings for context
from gensim.models import Word2Vec

data = [list(word) for word in words]

w2v = Word2Vec(sentences=data, vector_size=5, window=3, min_count=1, workers=1)

letter_vector = w2v.wv['a']
print("Vector for 'a':", letter_vector)

# model.save("letter_embeddings.model")
# Functions: Frequency, vowel proportion
from collections import defaultdict, Counter

def calculate_letter_frequencies(words):
    letter_frequencies = defaultdict(lambda: defaultdict(Counter))

    for word in words:
        word_length = len(word)
        for position, letter in enumerate(word):
            letter_frequencies[word_length][position][letter] += 1

    return letter_frequencies

frequencies = calculate_letter_frequencies(words)

import numpy as np

def calculate_vowel_consonant_probabilities(words):
    vowels = "aeiou"
    vowel_probabilities = defaultdict(lambda: defaultdict(Counter))
    consonant_probabilities = defaultdict(lambda: defaultdict(Counter))

    for word in words:
        word_length = len(word)
        for position, letter in enumerate(word):
            if letter in vowels:
                vowel_probabilities[word_length][position][letter] += 1
            else:
                consonant_probabilities[word_length][position][letter] += 1

    return vowel_probabilities, consonant_probabilities

vowel_probabilities, consonant_probabilities = calculate_vowel_consonant_probabilities(words)

def calculate_vowel_proportion(words):
    """
    Calculate the average proportion of vowels in a list of words.
    """
    vowels = "aeiou"
    total_vowels = total_letters = 0

    for word in words:
        total_vowels += sum(1 for letter in word if letter in vowels)
        total_letters += len(word)

    return total_vowels / total_letters if total_letters > 0 else 0
def calculate_letter_frequencies(words):
    """
    Calculate the frequency of each letter in each position across a list of words,
    normalized by the total occurrences of letters to mitigate the impact of rare words.
    """
    frequencies = defaultdict(lambda: defaultdict(int))
    total_counts = defaultdict(int)
    
    for word in words:
        for i, letter in enumerate(word):
            frequencies[i][letter] += 1
            total_counts[i] += 1

    # Normalize frequencies by total counts for each position
    for position in frequencies:
        for letter in frequencies[position]:
            frequencies[position][letter] /= total_counts[position]
    
    return frequencies

def calculate_letter_frequencies_with_length(words):
    """
    Calculate the frequency of each letter by its position in words of different lengths.
    """
    frequencies = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
    for word in words:
        word_length = len(word)
        for i, letter in enumerate(word):
            frequencies[word_length][i][letter] += 1
    return frequencies
# Statistical Prediction Function
def predict_missing_letter_with_context(pattern, guessed_letters, model, frequencies, words):
    vowels = "aeiou"
    vowel_proportion = calculate_vowel_proportion(words)  # Make sure this function is defined
    current_vowel_proportion = sum(1 for letter in pattern if letter in vowels) / len(pattern) if len(pattern) > 0 else 0
    best_guess, best_score = None, -np.inf
    guessed_vowels = set(guessed_letters) & set(vowels)
    word_length = len(pattern)
    
    # Decide whether to guess a vowel based on current vowel proportion
    guess_vowel = current_vowel_proportion < vowel_proportion and len(guessed_vowels) < len(vowels)

    for position, char in enumerate(pattern):
        if char == '_':
            candidates = frequencies[word_length][position] if word_length in frequencies and position in frequencies[word_length] else {}
            context_vector = None
            
            if position > 0 and pattern[position-1] != '_' and pattern[position-1] in model.wv:
                context_vector = model.wv[pattern[position-1]]
            if position < word_length - 1 and pattern[position+1] != '_' and pattern[position+1] in model.wv:
                next_vector = model.wv[pattern[position+1]]
                context_vector = (context_vector + next_vector) / 2 if context_vector is not None else next_vector
            
            for letter, _ in candidates.items():
                if letter in guessed_letters or (guess_vowel and letter not in vowels) or (not guess_vowel and letter in vowels):
                    continue

                embedding_similarity = np.dot(context_vector, model.wv[letter]) if context_vector is not None and letter in model.wv else 0
                frequency_score = np.log1p(candidates.get(letter, 0))
                combined_score = embedding_similarity + frequency_score
                
                if combined_score > best_score:
                    best_guess, best_score = letter, combined_score

    # If no suitable guess was found, especially in a vowel-needed situation, adjust strategy
    if best_guess is None and guess_vowel:
        for letter in model.wv.index_to_key:
            if letter not in guessed_letters and letter in vowels:
                return letter  # Return the first unguessed vowel as a fallback

    return best_guess

predict_missing_letter_with_context("cu_",['u','e','a','o','i','c','m','x'],w2v,frequencies, words)

Vector for 'a': [ 0.19897515 -0.0722477   0.0323585   0.01268803  0.5552395 ]


's'

# API

In [142]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = r'C:\Users\Fatima Azfar\Documents\assignment-gigs\hangman-ai\words_250000_train.txt'
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()

        self.current_dictionary = []
#         self.bigram_counts = bigram_counts_for_trigram  # Assuming it's defined globally or passed here
#         self.trigram_counts = trigram_counts  # Assuming it's defined globally or passed here
#         self.unigram_counts = unigram_counts  # Assuming it's defined globally or passed here
#         # self.trigram_counts = {}  # Placeholder for trigram counts
        # self.bigram_counts = {}  # Placeholder for bigram counts
        # self.unigram_counts = {}  # Placeholder for unigram counts

        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link
    
    # def guess(self, word):
    #     word = ''.join([letter if letter in self.guessed_letters else '_' for letter in word])
    #     words_by_length = load_words(r'C:\Users\Fatima Azfar\Documents\assignment-gigs\hangman-ai\words_250000_train.txt')
    #     word_length = len(word)
    #     if word_length not in words_by_length:
    #         return predict_missing_letter_with_context(word, self.guessed_letters, w2v, frequencies, words)
    #     else:
    #         possible_words = words_by_length[len(word)]
    #         return guess(possible_words, self.guessed_letters
        
    def guess(self, word):
        word = word.replace(" ", "")
        word = ''.join([letter if letter in self.guessed_letters else '_' for letter in word])
        if word.count('_') <= 0.8*(len(word)):
            word_with_missing = word.replace('_', '0')
            return predict_word(model, word_with_missing, char_to_int, int_to_char, max_word_length, self.guessed_letters)
        # words_by_length = list(load_words(r'C:\Users\Fatima Azfar\Documents\assignment-gigs\hangman-ai\words_250000_train.txt').keys())
        # word_length = len(word)
        # if word_length not in words_by_length:
        print("---------EMB---------")
        return predict_missing_letter_with_context(word, self.guessed_letters, w2v, frequencies, words)
        # else:
        #     print("---------COND---------")
        #     possible_words = load_words(r'C:\Users\Fatima Azfar\Documents\assignment-gigs\hangman-ai\words_250000_train.txt')[len(word)]
        #     # forward_probs, backward_probs = calculate_conditional_probabilities(possible_words)
        #     positional_frequencies = get_positional_letter_frequencies(possible_words, self.guessed_letters)
        #     # return get_best_guess_with_fallback(word, self.guessed_letters, forward_probs, backward_probs, positional_frequencies)
        #     return get_best_guess(positional_frequencies, self.guessed_letters)
        
    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

In [143]:
api = HangmanAPI(access_token="", timeout=2000)

In [144]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=1,verbose=True)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Successfully start a new game! Game ID: b3f7baaa863e. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
---------EMB---------
Guessing letter: i
Sever response: {'game_id': 'b3f7baaa863e', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ i _ _ _ _ _ _ _ '}
---------EMB---------
Guessing letter: e
Sever response: {'game_id': 'b3f7baaa863e', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ i _ _ _ _ _ _ _ '}
---------EMB---------
Guessing letter: a
Sever response: {'game_id': 'b3f7baaa863e', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ i _ a _ _ _ _ _ '}
LSTM
1/1 [==============================] - 0s 41ms/step
Guessing letter: m
Sever response: {'game_id': 'b3f7baaa863e', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ i _ a _ _ _ _ _ '}
LSTM
1/1 [==============================] - 0s 38ms/step
Guessing letter: b
Sever response: {'game_id': 'b3f7baaa863e', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ i _ a _ _ _ _ _ '}
LSTM
1/1 [===============

### LSTM MODEL BI.H5
- 29/100 for 30% LSTM and 70% EMB
- 38/100 for 40% LSTM and 60% EMB | 305/1000
- 35/100 for 50% LSTM and 50% EMB
- 30/100 for 60% LSTM and 40% EMB
- 28/100 for 70% LSTM and 30% EMB
- ####################################
- 33/100 for 40% LSTM and 60% COND
- 33/100 for 40% LSTM and 60% COND (100% one)
### LSTM MODEL BI 2.H5
### LSTM MODEL BI 3.H5 alone (LSTM 128)
- 41/100
- 320/1000
### LSTM MODEL BI 4.H5 alone (LSTM 256)